In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pickle
import mlflow
from mlflow.tracking import MlflowClient
import locale
import pytz
import os
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

#from utils.predictor import *

'es_ES.UTF-8'

In [2]:
import sklearn
import sklearn.preprocessing
print(sklearn.__version__)

1.2.2


In [4]:

#Cargar cliente mlfow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
client = MlflowClient()
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
tz_mexico = pytz.timezone('America/Mexico_City')

In [14]:
stations = ["MER", "UIZ"]
for station in stations:
    """
    # 15 min
    dir = r"C:\Users\valer\Documents\CIC\doctorado\Proyecto_Innovacion\Datos\air_traffic_"
    dir = dir+station+".csv"
    df = pd.read_csv(dir)
    df['traffic'] = df['traffic'].apply(lambda x: float(f"{x:.2f}"))
    df["date"] = df['date'].str.replace('/', '-')
    df["date"] = df["date"].str[:-9]
    # Traffic must be at the end
    #col = df["traffic"]
    #df.insert(len(df), col.name, col)
    """
    dir = dir[:-4]
    dir = dir+"_15m.csv"
    #df.to_csv(dir, index=False)

    # prom
    df = pd.read_csv(dir)
    #Si son negativos o vacios cambiarlos a nan
    for ind in range(df.shape[0]):
        for dato in df.columns:
            if(dato in ["CO", "NO",  "NOX","NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP", "traffic"]):
                if(df.loc[ind, dato] < 0):
                    df.loc[ind, dato] = np.nan
                    
                if(df.loc[ind, dato] == ""):
                    df.loc[ind, dato] = np.nan

    dir = dir[:-4]
    dir = dir+"_prom_hr.csv"
    df.to_csv(dir, index=False)

    
    df = pd.read_csv(dir)
    nuevoScaler = MinMaxScaler()
    #Obtenemos los nuevos escalers
    nuevoScaler.fit(df[["CO", "NO", "NOX", "NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP", "month", "hour"]])
    df_norm_data_escalada = df.copy()
    #Obtener los nuevos valores escalados
    df_norm_data_escalada[["CO", "NO", "NOX", "NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP", "month", "hour"]] = nuevoScaler.transform(df[["CO", "NO", "NOX", "NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP", "month", "hour"]])
    #scaler_dir = dir[:-8]+'scaler.pkl'
    scaler_dir = f"C:/Users/valer/Documents/CIC/doctorado/Proyecto_Innovacion/Datos/{station}_scaler.pkl"
    pickle.dump(nuevoScaler, open(scaler_dir, "wb"))
    dir2 = r"C:\Users\valer\Documents\CIC\doctorado\Proyecto_Innovacion\Datos\air_traffic_MER_norm.csv"
    df_norm_data_escalada = df_norm_data_escalada.rename(columns={'minutes': 'minute'})
    df_norm_data_escalada[['CO', 'NO', 'NOX', 'NO2', 'O3', 'PM10', 'PM25', 'RH', 'SO2','TMP', 'WDR', 'WSP']] = df_norm_data_escalada[['CO', 'NO', 'NOX', 'NO2', 'O3', 'PM10', 'PM25', 'RH', 'SO2','TMP', 'WDR', 'WSP']].round(12)
    df_norm_data_escalada = df_norm_data_escalada.where(pd.notnull(df_norm_data_escalada), "nan")
    df_norm_data_escalada.to_csv(dir2, index=False)
    #"""
    dir = r"C:\Users\valer\Documents\CIC\doctorado\Proyecto_Innovacion\Datos\air_and_traffic_"
    dir = dir+station+"_prom.csv"
    df = pd.read_csv(dir)
    df = df.where(pd.notnull(df), "nan")
    #df= df.rename(columns={'minutes': 'minute'})
    df.to_csv(dir, index=False)
    dir = r"C:\Users\valer\Documents\CIC\doctorado\Proyecto_Innovacion\Datos\air_and_traffic_"
    dir = dir+station+"_15m.csv"
    df = pd.read_csv(dir)
    df = df.where(pd.notnull(df), "nan")
    #df= df.rename(columns={'minutes': 'minute'})
    df.to_csv(dir, index=False)
    #"""

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 28-29: truncated \UXXXXXXXX escape (3859357940.py, line 14)

In [11]:
dir = r"C:\Users\valer\Documents\CIC\doctorado\Proyecto_Innovacion\Datos\air_traffic_UIZ"
dir = dir+"_prom.csv"
#df.to_csv(dir, index=False)
station = "UIZ"
df = pd.read_csv(dir)
nuevoScaler = MinMaxScaler()
#Obtenemos los nuevos escalers
nuevoScaler.fit(df[["CO", "NO", "NOX", "NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP"]])
df_norm_data_escalada = df.copy()
#Obtener los nuevos valores escalados
df_norm_data_escalada[["CO", "NO", "NOX", "NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP"]] = nuevoScaler.transform(df[["CO", "NO", "NOX", "NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP"]])
#scaler_dir = dir[:-8]+'scaler.pkl'
scaler_dir = f"C:/Users/valer/Documents/CIC/doctorado/Proyecto_Innovacion/Datos/{station}_scaler.pkl"
pickle.dump(nuevoScaler, open(scaler_dir, "wb"))

In [ ]:
model_name = "O3-"+str(station.lower())+"_24hr_forecast_model"
best_model_alias = "champion"

best_model_info = client.get_model_version_by_alias(model_name, best_model_alias)
best_model_run_id = best_model_info.run_id

scaler_dir = 'artifacts/'+station.upper()+'_scaler.pkl'
local_path = mlflow.artifacts.download_artifacts(run_id=best_model_run_id, artifact_path=scaler_dir)

#Subir a modelo 24hr
with mlflow.start_run(run_id=best_model_run_id) as run:
    mlflow.log_artifact(f'ML/Scalers/{station}_scaler.pkl', artifact_path="artifacts")

#Subir a modelo 1hr
model_name = "O3-"+str(station.lower())+"_1hr_forecast_model"
best_model_alias = "champion"

best_model_info = client.get_model_version_by_alias(model_name, best_model_alias)
best_model_run_id = best_model_info.run_id

with mlflow.start_run(run_id=best_model_run_id) as run:
    mlflow.log_artifact(f'ML/Scalers/{station}_scaler.pkl', artifact_path="artifacts")


# Changes

In [ ]:
#utils prediction
    df = df.dropna().reset_index(drop=True)

In [ ]:
#utils request_data_15m
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
tz_mexico = pytz.timezone('America/Mexico_City')

conocenos

In [ ]:
{% extends 'core/base.html' %}
{% block title %}Conocenos {% endblock %}

{%block background%}{%load static%}{%static 'core/img/portfolio-bg.jpg'%}{%endblock%}

{% block content %}
<div class="col-lg-3 col-md-4 offset-lg-1">
  <img class="img-fluid avatar" src="{%static 'core/img/AnaSaldaña.jpg' %}" alt="">
</div>
<div class="col-lg-7 col-md-8" style="text-align: justify; color: black;">
  <h2 class="section-heading" style="color: black; font-size: 24px !important;">Ana M. Magdalena Saldaña Pérez</h2>
  <h4 class="section-heading" style="color: black; font-size: 18px !important;">Académico responsable.</h4>    
  <p style="color: black;">Adscrita al Laboratorio de Procesamiento Inteligente de Información Geoespacial</p>
  <p><a href=https://www.cic.ipn.mx/index.php/ana-maria-magdalena-saldana>Directorio CIC</a></p>
</div>
  <div class="col-lg-3 col-md-4 offset-lg-1">
    <img class="img-fluid avatar" src="{%static 'core/img/valeria.jpg' %}" alt="">
  </div>
  <div class="col-lg-7 col-md-8" style="text-align: justify; color: black;">
    <h2 class="section-heading" style="color: black; font-size: 24px !important;">Valeria Legaria</h2>
    <p style="color: black;">Estudiante de doctorado en Ciencias de la Computación, en el Centro de Investigación en Computación del Instituto Politécnico Nacional con interés por el medio ambiente y la inteligencia artificial.</p>
    <p><a href=https://www.linkedin.com/in/valeria-legaria>LinkedIn</a></p>
    <p><a href=https://github.com/vlegaria>Github</a></p>
  </div>
  <div class="row">
    <div class="col-lg-3 col-md-4 offset-lg-1">
      <img class="img-fluid avatar" src="{%static 'core/img/francisco.jpg' %}" alt="">
    </div>
    <div class="col-lg-7 col-md-8" style="text-align: justify; color: black;">
      <h2 class="section-heading" style="color: black; font-size: 24px !important;">Francisco Ramírez</h2>
      <p style="color: black;">Estudiante de Ingeniería Mecatrónica, la Unidad Profesional Interdisciplinaria en Ingeniería y Tecnologías Avanzadas (UPIITA) del Instituto Politécnico Nacional. Con habilidades en programación y electrónica.</p>
      <p><a href=https://www.linkedin.com/in/francisco-ramirez-leon-203a13161>LinkedIn</a></p>
      <p><a href=https://github.com/FrankXav>Github</a></p>
    </div>
  </div>
  
  <div class="col-lg-3 col-md-4 offset-lg-1  ">
    <img class="img-fluid avatar" src="{%static 'core/img/RafaelRojas.jpg' %}" alt="">
  </div>
  <div class="col-lg-7 col-md-8" style="text-align: justify; color: black;">
    <h2 class="section-heading" style="color: black; font-size: 24px !important;">Rafael Rojas</h2>
    <p style="color: black;">Estudiante de Ing. Mecatrónica con amplia experiencia en el ámbito empresarial y apasionado por las ciencias computacionales, actualmente se desarrolla en el campo del ML y la Inteligencia Artificial</p>
    <p><a href=https://www.linkedin.com/in/rafael-y-rojas-nieves-839881166>LinkedIn</a></p>
    <p><a href=https://github.com/JFRo57/Portafolio>Github</a></p>
  </div>
  <div class="row">
    <div class="col-lg-3 col-md-4 offset-lg-1">
      <img class="img-fluid avatar" src="{%static 'core/img/Yoqsan.jpeg' %}" alt="">
    </div>
  <div class="col-lg-7 col-md-8" style="text-align: justify; color: black;">
  <h2 class="section-heading" style="color: black; font-size: 24px !important;">Yoqsan Angeles</h2>
    <p style="color: black;">Estudiante de doctorado en Ciencias de la Computación en el Instituto Politécnico Nacional de México, con trabajos en en desarrollo de algoritmos de optimización bioinspirados así como en el uso de redes neuronales aplicados en áreas como clasificación de patrones y robótica.</p>
    <p><a href=https://www.linkedin.com/in/yoqsan-angeles-ba8769244>LinkedIn</a></p>
  </div>
  

{% endblock %}




In [1]:

import locale
import mlflow
from mlflow.tracking import MlflowClient
import pytz
from utils.predictor import *

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
tz_mexico = pytz.timezone('America/Mexico_City')

#Cargar cliente mlfow
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
client = MlflowClient()

In [2]:
station = "mer"
model_name = "O3-"+str(station.lower())+"_24hr_forecast_model"
best_model_alias = "champion"

best_model_info = client.get_model_version_by_alias(model_name, best_model_alias)
print(best_model_info)
best_model_run_id = best_model_info.run_id
print(best_model_run_id)
scaler_dir = 'artifacts/'+station.upper()+'_scaler_O3.pkl'
print(scaler_dir)
local_path = mlflow.artifacts.download_artifacts(run_id=best_model_run_id, artifact_path=scaler_dir)

with open(local_path, "rb") as f:
    scaler = pickle.load(f)

<ModelVersion: aliases=['champion'], creation_timestamp=1738086504124, current_stage='None', description='', last_updated_timestamp=1738086504124, name='O3-mer_24hr_forecast_model', run_id='9ea9f70e7d9142f29645b791bf5f848b', run_link='', source=('mlflow-artifacts:/171761569910041749/9ea9f70e7d9142f29645b791bf5f848b/artifacts/mer '
 'station model for O3-24hr forecasting'), status='READY', status_message='', tags={'historicalData': '24'}, user_id='', version='1'>
9ea9f70e7d9142f29645b791bf5f848b
artifacts/MER_scaler_O3.pkl


NameError: name 'pickle' is not defined

In [48]:
from datetime import datetime, date, timedelta
import locale
import pytz
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
tz_mexico = pytz.timezone('America/Mexico_City')
stations2forecast = ["MER", "UIZ"]
if True:
    NOX = None
    datetime_now = datetime.now(tz_mexico)
    year = str(datetime_now.year)
    date_df = datetime_now.strftime('%Y-%m-%d')
    year = str(datetime_now.year)
    month = str(datetime_now.month)
    day = str(datetime_now.day)
    hour = str(datetime_now.hour)
    minute= str(datetime_now.minute)

    #Preparar datos para buscar tabla
    dayText = f'0{day}' if len(str(day)) == 1 else str(day)
    monthText = f'0{month}' if len(str(month))  == 1 else str(month)
    dateData = f'{dayText}-{monthText}-{year}'
    
    claveCont = ['co','no','nox','no2','o3','pm10','pm2','rh','so2','tmp','wdr','wsp']

    #Crear dataframe vacio por estacion

    contEstacion = pd.DataFrame(columns=["station","date", "CO", "NO", "NOX","NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP", "year", "month", "day", "hour", "minutes", "traffic"])

    for station in stations2forecast:
        contEstacion = pd.concat([contEstacion, pd.DataFrame({"station":station,
                                                "date":datetime_now,
                                                "year":year,
                                                "month":month,
                                                "day":day,
                                                "hour":hour,
                                                "minutes":"0"}, index=[0])], ignore_index=True)

    #print(contEstacion.head())

    for cont in claveCont:
        urlGob = f"http://www.aire.cdmx.gob.mx/estadisticas-consultas/concentraciones/respuesta.php?qtipo=HORARIOS&parametro={cont}&anio={year}&qmes={month}"
        
        #print(urlGob)
        df_consulta = pd.read_html(urlGob, encoding='windows-1252')

        df_datos = df_consulta[0]

        df_datos.columns = df_datos.iloc[1]

        df_datos = df_datos.drop([0,1])

        #print(df_datos.head())

        #Nombre de la culmna del contaminante en el dataframe de estaciones
        if(cont == "pm2"):
            colEstation = "PM25"
        else: 
            colEstation = cont.upper()


        for station in stations2forecast:
            dataofDay = df_datos.loc[df_datos['Fecha'] == dateData, ["Hora",station]]

            dato = dataofDay.loc[dataofDay['Hora'] == hour, station].values[0]

            if(dato != "nr"):
                if float(dato) >= 0:
                    contEstacion.loc[contEstacion['station'] == station, colEstation] = dato
                else: 
                    contEstacion.loc[contEstacion['station'] == station, colEstation] = np.nan
            else: 
                contEstacion.loc[contEstacion['station'] == station, colEstation] = np.nan
    print("Datos gob")
    #print(contEstacion.tail())
    #"CO", "NO", "NOX","NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP", "year", "month", "day", "hour", "minutes", "traffic"])
    
    #df['traffic'] = df['traffic'].apply(lambda x: float(f"{x:.2f}"))
    #"""
    CO =contEstacion.loc[contEstacion['station'] == station,"CO"].values[0]
    NO = contEstacion.loc[contEstacion['station'] == station,"NO"].values[0]
    NOX = contEstacion.loc[contEstacion['station'] == station,"NOX"].values[0]
    NO2= contEstacion.loc[contEstacion['station'] == station,"NO2"].values[0]
    O3= contEstacion.loc[contEstacion['station'] == station,"O3"].values[0]
    PM10 =contEstacion.loc[contEstacion['station'] == station,"PM10"].values[0]
    PM25 = contEstacion.loc[contEstacion['station'] == station,"PM25"].values[0]
    RH = contEstacion.loc[contEstacion['station'] == station,"RH"].values[0]
    SO2= contEstacion.loc[contEstacion['station'] == station,"SO2"].values[0]
    TMP = contEstacion.loc[contEstacion['station'] == station,"TMP"].values[0]
    WDR = contEstacion.loc[contEstacion['station'] == station,"WDR"].values[0]
    WSP= contEstacion.loc[contEstacion['station'] == station,"WSP"].values[0]
    print(CO, NO, NOX,NO2, O3, PM10, PM25, RH, SO2, TMP, WDR, WSP)
            

IndexError: index 0 is out of bounds for axis 0 with size 0

In [52]:
if len(dataofDay.loc[dataofDay['Hora'] == hour, station]) == 0:
    print("vacio")


vacio


In [47]:
station = "MER"

for cont in claveCont:
    urlGob = f"http://www.aire.cdmx.gob.mx/estadisticas-consultas/concentraciones/respuesta.php?qtipo=HORARIOS&parametro={cont}&anio={year}&qmes={month}"
    
    #print(urlGob)
    df_consulta = pd.read_html(urlGob, encoding='windows-1252')

    df_datos = df_consulta[0]

    df_datos.columns = df_datos.iloc[1]

    df_datos = df_datos.drop([0,1])

    #print(df_datos.head())

    #Nombre de la culmna del contaminante en el dataframe de estaciones
    if(cont == "pm2"):
        colEstation = "PM25"
    else: 
        colEstation = cont.upper()


    for station in stations2forecast:
        dataofDay = df_datos.loc[df_datos['Fecha'] == dateData, ["Hora",station]]

        print(cont,station)
        #print(dataofDay)
        #float(dataofDay.loc[dataofDay['Hora'] == hour, station].values[0])
        dato = dataofDay.loc[dataofDay['Hora'] == hour, station].values[0]
        if(dato != "nr"):
            if float(dato) >= 0:
                contEstacion.loc[contEstacion['station'] == station, colEstation] = dato
            else: 
                contEstacion.loc[contEstacion['station'] == station, colEstation] = np.nan
        else: 
            contEstacion.loc[contEstacion['station'] == station, colEstation] = np.nan
print("Datos gob")
print(contEstacion.tail())

co MER
1   Hora   MER
674    1    nr
675    2    nr
676    3    nr
677    4  0.72
678    5  0.62
679    6  0.82
680    7  1.36
681    8  2.46
682    9  2.00
683   10  1.22
684   11  0.92
685   12  0.79
686   13    nr
687   14  0.61
co UIZ
1   Hora   UIZ
674    1    nr
675    2    nr
676    3    nr
677    4  0.60
678    5  0.56
679    6  0.73
680    7  1.24
681    8  1.64
682    9  2.48
683   10  1.60
684   11  1.76
685   12  1.05
686   13  0.81
687   14  0.48
no MER
1   Hora  MER
674    1   nr
675    2   nr
676    3   nr
677    4    8
678    5   10
679    6   21
680    7   67
681    8  100
682    9   93
683   10   32
684   11   13
685   12   11
686   13   nr
687   14    4
no UIZ
1   Hora  UIZ
674    1   nr
675    2   nr
676    3   nr
677    4    2
678    5    1
679    6    3
680    7   21
681    8   58
682    9  136
683   10   39
684   11   33
685   12   12
686   13    5
687   14    2
nox MER
1   Hora  MER
674    1   nr
675    2   nr
676    3   nr
677    4   48
678    5   50
679    6  

In [35]:

        try:

            datetime_now = datetime.now(tz_mexico)
            year = str(datetime_now.year)
            date_df = datetime_now.strftime('%Y-%m-%d')
            year = str(datetime_now.year)
            month = str(datetime_now.month)
            day = str(datetime_now.day)
            hour = str(datetime_now.hour)
            minute= str(datetime_now.minute)

            #Preparar datos para buscar tabla
            dayText = f'0{day}' if len(str(day)) == 1 else str(day)
            monthText = f'0{month}' if len(str(month))  == 1 else str(month)
            dateData = f'{dayText}-{monthText}-{year}'
            
            claveCont = ['co','no','nox','no2','o3','pm10','pm2','rh','so2','tmp','wdr','wsp']

            #Crear dataframe vacio por estacion

            contEstacion = pd.DataFrame(columns=["station","date", "CO", "NO", "NOX","NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP", "year", "month", "day", "hour", "minutes", "traffic"])

            for station in stations2forecast:
                contEstacion = pd.concat([contEstacion, pd.DataFrame({"station":station,
                                                        "date":datetime_now,
                                                        "year":year,
                                                        "month":month,
                                                        "day":day,
                                                        "hour":hour,
                                                        "minutes":"0"}, index=[0])], ignore_index=True)
        
            #print(contEstacion.head())

            for cont in claveCont:
                urlGob = f"http://www.aire.cdmx.gob.mx/estadisticas-consultas/concentraciones/respuesta.php?qtipo=HORARIOS&parametro={cont}&anio={year}&qmes={month}"
                
                #print(urlGob)
                df_consulta = pd.read_html(urlGob, encoding='windows-1252')

                df_datos = df_consulta[0]

                df_datos.columns = df_datos.iloc[1]

                df_datos = df_datos.drop([0,1])

                #print(df_datos.head())

                #Nombre de la culmna del contaminante en el dataframe de estaciones
                if(cont == "pm2"):
                    colEstation = "PM25"
                else: 
                    colEstation = cont.upper()


                for station in stations2forecast:
                    dataofDay = df_datos.loc[df_datos['Fecha'] == dateData, ["Hora",station]]
                    dato = dataofDay.loc[dataofDay['Hora'] == hour, station]
                    print(dato)
                    dato = dato.values[0]

                    if(dato != "nr"):
                        if float(dato) >= 0:
                            contEstacion.loc[contEstacion['station'] == station, colEstation] = dato
                        else: 
                            contEstacion.loc[contEstacion['station'] == station, colEstation] = np.nan
                    else: 
                        contEstacion.loc[contEstacion['station'] == station, colEstation] = np.nan
            print("Datos gob")
            print(contEstacion.tail())
            #"CO", "NO", "NOX","NO2", "O3", "PM10", "PM25", "RH", "SO2", "TMP", "WDR", "WSP", "year", "month", "day", "hour", "minutes", "traffic"])
                
            CO =contEstacion.loc[contEstacion['station'] == station,"CO"].values[0]
            NO = contEstacion.loc[contEstacion['station'] == station,"NO"].values[0]
            NOX = contEstacion.loc[contEstacion['station'] == station,"NOX"].values[0]
            NO2= contEstacion.loc[contEstacion['station'] == station,"NO2"].values[0]
            O3= contEstacion.loc[contEstacion['station'] == station,"O3"].values[0]
            PM10 =contEstacion.loc[contEstacion['station'] == station,"PM10"].values[0]
            PM25 = contEstacion.loc[contEstacion['station'] == station,"PM25"].values[0]
            RH = contEstacion.loc[contEstacion['station'] == station,"RH"].values[0]
            SO2= contEstacion.loc[contEstacion['station'] == station,"SO2"].values[0]
            TMP = contEstacion.loc[contEstacion['station'] == station,"TMP"].values[0]
            WDR = contEstacion.loc[contEstacion['station'] == station,"WDR"].values[0]
            WSP= contEstacion.loc[contEstacion['station'] == station,"WSP"].values[0]
            print("GOB")
            print(CO, NO, NO2, NOX,O3, PM10, PM25, RH, SO2, TMP, WDR, WSP)
        except:

'14'

In [ ]:

        min_val = loaded_scaler.data_min_[4]  # Valor mínimo del O3
        max_val = loaded_scaler.data_max_[4]  # Valor máximo del O3
        # Aplicar la transformación inversa 
        predicciones = predicciones_normalizadas * (max_val - min_val) + min_val
        #y_test_normalizadas = y_test.reshape(-1, 1)
        #y_test = loaded_scaler.inverse_transform(y_test_normalizadas)
        y_test = y_test * (max_val - min_val) + min_val